In [32]:
import numpy as np
import plotly.express as px

In [33]:
def T(d = 0, thetta = 0, a = 0, alpha = 0):
    return np.array([[np.cos(thetta), -np.sin(thetta)*np.cos(alpha),  np.sin(thetta)*np.sin(alpha), a*np.cos(thetta)],
                     [np.sin(thetta),  np.cos(thetta)*np.cos(alpha), -np.cos(thetta)*np.sin(alpha), a*np.sin(thetta)],
                     [             0,                 np.sin(alpha),                 np.cos(alpha),                d],
                     [             0,                             0,                             0,                1]])

In [34]:
def FK(d, thetta, a, alpha):
    Temp_t = np.eye(4)

    # Вектор для определения координат джоинтов
    p_0 = np.array([0, 0, 0, 1])

    # Координаты джоинтов
    joint = np.array([[0, 0, 0]])

    # Определение положения джоинтов через матрицы однородных преобразований
    for i in range(0,3):
        Temp_t = Temp_t @ T(d[i], thetta[i], a[i], alpha[i])
        coordinate = (Temp_t @ p_0)
        joint = np.vstack([joint, coordinate[0:3]])

    # Положение эндефектора
    endefector_pos = [[joint[3][0]], [joint[3][1]], [joint[3][2]]]

    return joint, endefector_pos

In [35]:
def plotting(x, y, z):
    fig = px.line_3d(x=x, y=y, z=z, markers=True, range_x=[-30,30], range_y=[-30,30], range_z=[0,30])
    fig.show()

In [36]:
# Обобщенные координаты
d1 = 15
q1 = 30
q2 = -30

l2 = 15
l3 = 12

# Полные параметры
d = [d1, 0, 0]
thetta = [90, q1, -q2]
a = [0, l2, l3]
alpha = [0, 0, 0]
thetta = np.deg2rad(thetta)

In [37]:
# Решение прямой кинематики
joint, endefector = FK(d, thetta, a, alpha)

# Вычисление матрицы преобразования
T03 = T(d[0], thetta[0], a[0], alpha[0])
for i in range(1,3):
  T03 = T03 @ T(d[i], thetta[i], a[i], alpha[i])
# Выделение матрицы поворота
R03 = T03[:3,:3]

# Расчет уголв Эйлера для комбинации ZYZ
a = np.arctan2(R03[1][2], R03[0][2])
b = np.arctan2(np.sqrt(1-R03[2][2]**2), R03[2][2])
c = np.arctan2(R03[2][1], -R03[2][0])

plotting(joint.T[0], joint.T[1], joint.T[2])
print("Положение эндефектора: x =", endefector[0], "y =", endefector[1], "z =", endefector[2])
print("Положение эндефектора: a =", np.rad2deg(a), "b =", np.rad2deg(b), "c =", np.rad2deg(c))

Положение эндефектора: x = [-17.89230484541326] y = [18.99038105676658] z = [15.0]
Положение эндефектора: a = 0.0 b = 0.0 c = 180.0


### Обратная кинематика

In [39]:
x = -17.8923
y = 18.99

L = np.sqrt(x**2 + y**2)
thetta2 = np.arccos((L**2 - l2**2 - l3**2)/(2*l2*l3))
alpha = np.arctan2(y,x)
betta = np.arccos((L**2+l2**2-l3**2)/(2*l2*L))
thetta1 = alpha - betta
thetta1 = thetta1 - np.pi/2

print(thetta1*180/np.pi, thetta2*180/np.pi)

29.998512587497515 30.004661661177963
